# Named-entity Recognition (NER) 

Named entity recognition is a fundamental task in information extraction from textual documents. While named entities originally corresponded to real-world entities with names (named entities), this concept has been extended to any type of information: it is possible to extract chemical molecules, product numbers, amounts, addresses, etc. In this practical assignment, we will use several named entity extraction libraries in French on a small corpus. The objective is not to train the best possible model, but to test the use of each of these libraries.



## The AdminSet dataset
The AdminSet dataset is a corpus of administrative documents in French produced by automatic character recognition and manually annotated with named entities. This corpus is quite difficult because the document recognition process produces noisy text (errors due to layout, recognition, fonts, etc.).

The paper describing the dataset is available [here](https://hal.science/hal-04855066v1/file/AdminSet_et_AdminBERT__version___preprint.pdf).

The corpus is available on HuggingFace: [Adminset-NER](https://huggingface.co/datasets/taln-ls2n/Adminset-NER).

In [1]:
from datasets import load_dataset
ds = load_dataset('taln-ls2n/Adminset-NER')
print(ds)

/home/onyxia/work/nlpenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating validation split: 100%|██████████| 85/85 [00:00<00:00, 10944.80 examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 729
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 85
    })
})


#### Question
> * Compute descriptive statistics on the texts  for each split (train, dev)
> * Compute descriptive statistics on the entities for each split (train, dev)
> * Compare with the statistics reported in the paper (Table 2)
> * Display a couple of random texts with their entities

In [2]:
# Compute statistics on the number of token in train and validation : min, max, mean std, median
import numpy as np
from collections import Counter
import random
import pandas as pd

In [3]:
temp = ds["train"].map(lambda x: {"length": len(x["tokens"])})
temp["length"]

Map: 100%|██████████| 729/729 [00:00<00:00, 8906.82 examples/s]


Column([63, 24, 31, 18, 41, ...])

In [4]:
t = temp["length"]
print(min(t), max(t), np.mean(t), np.std(t), np.median(t))

15 379 63.3676268861454 52.66968106175962 45.0


In [5]:
np.sum(temp["length"])

np.int64(46195)

In [6]:
temp = ds["train"].map(lambda x: {
    "length": len(x["ner_tags"]), 
    "nb_ORG": x["ner_tags"].count("B-ORG")+x["ner_tags"].count("I-ORG"),
    "nb_PER": x["ner_tags"].count("B-PER")+x["ner_tags"].count("I-PER"),
    "nb_LOC": x["ner_tags"].count("B-LOC")+x["ner_tags"].count("I-LOC")
})
temp["nb_PER"]

Map: 100%|██████████| 729/729 [00:00<00:00, 8614.28 examples/s]


Column([3, 5, 5, 2, 2, ...])

In [7]:
t = temp["nb_ORG"]
print(min(t), max(t), np.mean(t), np.std(t), np.median(t), np.sum(t))

0 35 3.0809327846364885 4.816555948106844 1.0 2246


In [8]:
list(zip(ds["train"][0]["tokens"],ds["train"][0]["ner_tags"]))

[('fin', 'O'),
 ('Procès-Verbal', 'O'),
 ('Conseil', 'O'),
 ('communautaire', 'O'),
 ('du', 'O'),
 ('lundi', 'O'),
 ('19', 'O'),
 ('juin', 'O'),
 ('2023', 'O'),
 ("L'an", 'O'),
 ('deux', 'O'),
 ('mille', 'O'),
 ('vingt-trois', 'O'),
 (',', 'O'),
 ('le', 'O'),
 ('lundi', 'O'),
 ('19', 'O'),
 ('juin', 'O'),
 (',', 'O'),
 ('à', 'O'),
 ('18', 'O'),
 ('heures', 'O'),
 ('30', 'O'),
 (',', 'O'),
 ('le', 'O'),
 ('conseil', 'O'),
 ('communautaire', 'O'),
 ("s'est", 'O'),
 ('réuni', 'O'),
 ('à', 'O'),
 ('Coucy', 'B-LOC'),
 ('le', 'I-LOC'),
 ('Château', 'I-LOC'),
 ('conformément', 'O'),
 ('à', 'O'),
 ("l'article", 'O'),
 ('2122-17', 'O'),
 ('du', 'O'),
 ('Code', 'O'),
 ('général', 'O'),
 ('des', 'O'),
 ('Collectivités', 'O'),
 ('Territoriales', 'O'),
 ('sur', 'O'),
 ('la', 'O'),
 ('convocation', 'O'),
 ('de', 'O'),
 ('Monsieur', 'B-PER'),
 ('Vincent', 'I-PER'),
 ('MORLET', 'I-PER'),
 (',', 'O'),
 ('Président', 'O'),
 (',', 'O'),
 ('adressée', 'O'),
 ('aux', 'O'),
 ('délégués', 'O'),
 ('des', 'O')

In [32]:
ds_train.num_rows

583

In [ ]:
# for item in ds_train

### Creation of the splits

The train_test_split() function from huggingface allow to split a dataset randomly in 2 parts : https://huggingface.co/docs/datasets/v4.5.0/process#split

The ```spacy_utils.py``` file contains functions to save a dataset in text format (```save_text```, usefull for inspection), BIO format (```save_bio```) and spacy format (```save_docbin```).

#### Questions
>* Using the split function, create a train/dev/test split corresponding to the proportions reported in the paper
>* Save the sets in a corpus directory, in text, bio and docbin formats.

In [23]:
from spacy_utils import save_bio, save_text, save_docbin
from spacy.tokens import Doc, DocBin
import spacy

In [28]:
temp = ds["train"].train_test_split(test_size=0.2)
ds_train = temp["train"]
ds_dev = temp["test"]
ds_test = ds["validation"]

In [35]:
import spacy_utils
def aux(d,output_path):
    save_text(d,"corpus/"+output_path+".txt")
    save_bio(d,"corpus/"+output_path+".bio")
    save_docbin(d,"corpus/"+output_path+".docbin")
aux(ds_train,"train")
aux(ds_dev,"dev")
aux(ds_test,"test")

Saving text to corpus/train.txt...


100%|██████████| 583/583 [00:00<00:00, 3163.34it/s]


Saved to corpus/train.txt
Saving BIO text to corpus/train.bio...


100%|██████████| 583/583 [00:00<00:00, 4503.67it/s]


Saved to corpus/train.bio
Creating corpus/train.docbin with 583 examples...


...

Saved to corpus/test.txt
Saving BIO text to corpus/test.bio...


100%|██████████| 85/85 [00:00<00:00, 5178.30it/s]


Saved to corpus/test.bio
Creating corpus/test.docbin with 85 examples...


100%|██████████| 85/85 [00:00<00:00, 797.42it/s]

Saved to corpus/test.docbin


### Testing spaCy pre-trained NER models

spaCy comes with a several pretrained models for many languages. For French, 4 models are provided : https://spacy.io/models/fr

To apply a pretrained model to dataset, use : 
- ```nlp = spacy.load(MODEL_NAME)``` to load the model. You need to download it first with "spacy download MODEL_NAME"
- ```DocBin().from_disk()``` to load a dataset in spaCy format from the disk
- ```doc_bin.get_docs(nlp.vocab)``` to convert the dataset from binary to text format
- ```nlp(doc.text)```to apply the NER model to a text

To evaluate the prediction, you can use the spaCy [Scorer](https://spacy.io/api/scorer)
- ```scorer.score(examples)``` where examples is a list of spaCy ```Example(prediction, reference)````

#### Question

>* Using a spaCy pretrained model for French, evaluate its performace for NER prediction on the train, dev and test sets
>* Compare this model to results reported in the paper

In [37]:
import spacy
from spacy.tokens import DocBin
from spacy.scorer import Scorer
from spacy.training import Example
from tqdm import tqdm
from prettytable import PrettyTable # optional but nice

In [51]:
# python -m spacy download fr_core_news_sm
nlp = spacy.load('fr_core_news_sm')
doc_bin = DocBin().from_disk("corpus/train.docbin")
doc_list = list(doc_bin.get_docs(nlp.vocab))

In [84]:
type(doc_list[0].text)

str

In [85]:
toto = nlp(doc_list[0].text)

In [86]:
for token in toto:
    print(token.text, token.lemma_, token.ent_type_)

Il il 
développe développer 
des un 
actions action 
et et 
des un 
projets projet 
autour autour 
de de 
l' le 
image image 
, , 
du de 
développement développement 
durable durable 
, , 
de de 
l' le 
aménagement aménagement 
urbain urbain 
, , 
du de 
patrimoine patrimoine 
, , 
etc. etc. 
Le le ORG
Comité comité ORG
engage engager 
notamment notamment 
des un 
actions action 
de de 
coopération coopération 
décentralisée décentraliser 
entre entrer 
Angoulême angoulême LOC
et et 
Ségou Ségou LOC
Mali Mali LOC
, , 
contribuant contribuer 
ainsi ainsi 
au au 
rayonnement rayonnement 
international international 
de de 
la le LOC
Ville ville LOC
. . 


In [87]:
for (pred,ref) in zip(toto,doc_list[0]):
    print(pred.text, pred.ent_type_, ref.ent_type_, sep="\t")

Il		
développe		
des		
actions		
et		
des		
projets		
autour		
de		
l'		
image		
,		
du		
développement		
durable		
,		
de		
l'		
aménagement		
urbain		
,		
du		
patrimoine		
,		ORG
etc.		ORG
Le	ORG	
Comité	ORG	
engage		
notamment		
des		
actions		
de		
coopération		
décentralisée		ORG
entre		
Angoulême	LOC	ORG
et		ORG
Ségou	LOC	
Mali	LOC	
,		
contribuant		
ainsi		
au		
rayonnement		
international		LOC
de		LOC
la	LOC	


In [90]:
predictions = list(map(lambda d: nlp(d.text), doc_list))

In [68]:
for token in predictions[0]:
    print(token.lemma_, token.ent_type_)

il 
développer 
un 
action 
et 
un 
projet 
autour 
de 
l'image 
, 
de 
développement 
durable 
, 
de 
l'aménagement 
urbain 
, 
de 
patrimoine 
, 
etc. 
le ORG
comité ORG
engager 
notamment 
un 
action 
de 
coopération 
décentraliser 
entrer 
angoulême ORG
et 
Ségou ORG
Mali ORG
, 
contribuer 
ainsi 
au 
rayonnement 
international 
de 
le LOC
ville LOC
. 


In [91]:
type(predictions[0])

spacy.tokens.doc.Doc

In [92]:
scorer = Scorer(nlp)

In [93]:
examples = list(map(Example, predictions,doc_list))

In [94]:
scorer.score(examples)

{'token_acc': None,
 'token_p': None,
 'token_r': None,
 'token_f': None,
 'pos_acc': 0.9411811367151866,
 'morph_acc': 0.9413118831529648,
 'morph_micro_p': 0.9524083734164352,
 'morph_micro_r': 0.9696854738584633,
 'morph_micro_f': 0.9609692744112923,
 'morph_per_feat': {'Gender': {'p': 0.9605792437650845,
   'r': 0.968931138418734,
   'f': 0.964737115484504},
  'Number': {'p': 0.9851409389346892,
   'r': 0.9802053146028783,
   'f': 0.98266692928895},
  'Person': {'p': 0.904603068712475,
   'r': 0.900398406374502,
   'f': 0.9024958402662229},
  'Mood': {'p': 0.9162348877374784,
   'r': 0.9107296137339056,
   'f': 0.9134739560912613},
  'Tense': {'p': 0.9436201780415431,
   'r': 0.9228855721393034,
   'f': 0.9331377069796688},
  'VerbForm': {'p': 0.9548599670510708,
   'r': 0.9330328396651641,
   'f': 0.943820224719101},
  'Definite': {'p': 0.8876337693222355,
   'r': 0.992686170212766,
   'f': 0.9372253609541745},
  'PronType': {'p': 0.893644617380026,
   'r': 0.9891304347826086,
   

### Training a custom spaCy model

The training of a cupstom spaCy NER model can be done both with the command line interface (cli) or in a python script. Using the cli is ususally more optimzed. All the configuration of the training is defined in a coniguration file, which is a good practice for documentation, tracing and reproducibility.

The configuration file can be generated on line using the [Quickstart](https://spacy.io/usage/training#quickstart)

<img src="images/spacy_quickstart.jpg" width="600" >

You can run the training process as a script using the train function (https://spacy.io/usage/training#api-train), specifying the configuration file and the directory in which to save the model as parameters. Once the training is complete, the best and last models are saved in the directory.

#### Question
> * Generate a training configuration file for a NER in French
> * Add the correct path to the training and dev sets generated previously
> * train a NER model
> * Evaluate the model on the train, dev et test sets. Compare to the results reported in the paper.

In [ ]:
# train the model
from spacy.cli.train import train


In [ ]:
# evaluate the model

### Zero-shot NER prediction with GLiNER


[GLiNER](https://github.com/fastino-ai/GLiNER2/tree/main)  is a library that provides models for zero-shot named entity recognition. This means that[structured information extraction](https://github.com/fastino-ai/GLiNER2/blob/main/tutorial/3-json_extraction.md)structured information extraction, which means that the extracted information can be organised in a structured JSON format. GLiNER does not provide the location of entities in the text by default, but you can configure the model to output this information (```include_spans=True```). Finally, GLiNER enables entities to be overlapped and nested, which is not supported by the spaCy scorer. The spaCy [filter_spans](https://spacy.io/api/top-level#util.filter_spans) function can be used to remove overlapping entities for evaluation.

#### Question
> * Define the entities to extract from the text.
> * Apply GLiNER on the dev and test sets
> * Evaluate the models on the dev and test sets and compare to the results reported in the paper.

In [ ]:
from gliner2 import GLiNER2
extractor = GLiNER2.from_pretrained("fastino/gliner2-base-v1")
from spacy.util import filter_spans
nlp = spacy.blank("fr")  # tokenizer only

doc_bin = DocBin().from_disk("REPLACE")
gold_docs = list(doc_bin.get_docs(nlp.vocab))

label_map = {
    # Define the entities here
}

gliner_labels = list(label_map.values())
reverse_map = {v: k for k, v in label_map.items()}

examples = []

for gold_doc in gold_docs:
    text = gold_doc.text
    
    predictions = extractor.extract_entities(text, gliner_labels, include_spans=True)
    pred_doc = nlp.make_doc(text)

    spans = []
    for gliner_label, entities in predictions["entities"].items():
        spacy_label = reverse_map.get(gliner_label)
        if not spacy_label:
            continue
        for ent in entities:
            start = ent["start"]
            end = ent["end"]

            span = pred_doc.char_span(start, end, label=spacy_label)
            if span:
                spans.append(span)

    spans = filter_spans(spans)
    pred_doc.ents = spans
    examples.append(Example(pred_doc, gold_doc))

